# **Clustering des titres**

Le but de ce notebook est d'essayer de faire un clustering sur la colonne titres de notre fichier, a fin de pouvoir ensuite donner un thème par cluster 

In [13]:
# Importation des bibliothèques nécessaires
# pandas pour la manipulation des données 
# scikit-learn pour la création de la matrice tf-idf et le calcul de la similarité cosinus
# Dbscan pour le clustering
# nltk pour les stop words en français

import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
import numpy as np
from nltk.corpus import stopwords
import nltk

# Télécharger les stop words français si nécessaire
nltk.download('stopwords', quiet=True)
french_stop_words = stopwords.words('french')

In [14]:
# Chargement des données 
data= pd.read_csv("articles.csv")
# Nombre d'articles 
print (data.shape[0])
print (data.columns)
# Remplacer le nom de la première colonne par "id"
data.rename(columns={data.columns[0]: "id"}, inplace=True)
print (data.columns)

1269
Index(['Unnamed: 0', 'title', 'year', 'language', 'Nb_authors'], dtype='object')
Index(['id', 'title', 'year', 'language', 'Nb_authors'], dtype='object')


In [15]:
# Dans le fichier y'a des articles en français et en anglais 
data_fr = data[data['language'] == 'fr']
print (data_fr.shape[0])
data_en = data[data['language'] == 'en']
print (data_en.shape[0])

1133
120


In [16]:
# Récupération du vocabulaire français à partir des titres des articles

import spacy
from spacy.lang.fr.stop_words import STOP_WORDS as french_stop_words

# Charger le modèle français
nlp = spacy.load("fr_core_news_sm")

# Initialiser un set vide pour le vocabulaire
vocab_fr = set()

for title in data_fr['title']:
    doc = nlp(title)
    for token in doc:
        # garder seulement les mots alphabétiques, minuscules, non stopwords
        lemma = token.lemma_.lower()
        if token.is_alpha and lemma not in french_stop_words:
            vocab_fr.add(lemma)

print("Nombre de mots uniques :", len(vocab_fr))


Nombre de mots uniques : 1914


In [17]:
print(vocab_fr)

{'bit', 'nettoyage', 'gml', 'sequencesviewer', 'divergente', 'reglo', 'spécialiser', 'heuristique', 'exploitation', 'job', 'dépêche', 'expérimental', 'attribut', 'biodiversité', 'métadonnée', 'p', 'édition', 'snow', 'télévisées', 'structurel', 'supplémentaire', 'mesurer', 'réutilisation', 'coller', 'erreur', 'logique', 'streams', 'vision', 'clique', 'mayer', 'lexical', 'hôpital', 'centrer', 'fabrique', 'lexicosyntaxique', 'ensembliste', 'aberrant', 'particulier', 'sécuriser', 'médical', 'étoile', 'robuste', 'gouvernance', 'modulaire', 'fréquente', 'owl', 'kd', 'lexico', 'temporellement', 'profil', 'clusterings', 'thermique', 'architectural', 'récolte', 'interopérabilité', 'croyance', 'gramlab', 'carré', 'clos', 'structurellement', 'plainte', 'sémantique', 'situer', 'social', 'capacitaire', 'répétition', 'asymétrique', 'archéologique', 'glose', 'cabine', 'miner', 'index', 'alternative', 'perforecast', 'rdbtoonto', 'performance', 'connexionniste', 'technologie', 'donnée', 'haptique', 'te

In [42]:
# Nettoyage manuel des mots non pertinents
manual_stop_words = {
    # Bruit & Lettres isolées
    'p', 'n', 'b', 'c', 'j', 'v', 'm', 'r', 'k', 'cl', 'sou', 'tr', 'vs', 'via', 'multi', 'intro', 'chapitre',
    
    # Termes génériques de recherche
    'approche', 'approches', 'méthode', 'méthodes', 'système', 'systèmes',
    'application', 'applications', 'analyse', 'analyses', 'étude', 'études',
    'modèle', 'modèles', 'modélisation', 'algorithme', 'algorithmes',
    'problème', 'problèmes', 'solution', 'solutions', 'résultat', 'résultats',
    'outil', 'outils', 'processus', 'technique', 'techniques', 'concept',
    'contexte', 'cadre', 'travail', 'travaux',
    'contribution', 'contributions', 'cas', 'exemple', 'exemples', 'usage',
    'utilisation', 'proposition', 'démarche', 'principe', 'théorie', 'théorique',
    'pratique', 'état', 'art', 'vue', 'niveau', 'type', 'moyen', 'question',
    'enjeu', 'domaine', 'sujet', 
    
    # Verbes courants (formes lemmatisées probables)
    'utiliser', 'permettre', 'baser', 'proposer', 'présenter','définir', 'construire', 'générer', 'traiter',
    'extraire', 'apprendre',  'considérer', 'mesurer', 'voir',
    'faire', 'obtenir','viser', 'fournir', 'intégrer',
    'mettre', 'partir',  'montrer', 'conclure',
    
    # Adjectifs / Adverbes vagues
    'nouveau', 'nouvelle', 'nouveaux', 'nouvelles', 'bon', 'meilleur',
    'grand', 'petit', 'simple',
    'général', 'efficace', 'performant', 'rapide',  'classique', 'possible',
    'nécessaire', 'principal', 'important', 'haut', 'faible', 'large',
    'court', 'long', 'très', 'trop', 'peu', 'bien', 'mieux'
}
vocab_fr = vocab_fr - manual_stop_words
print(len(vocab_fr))

1822


In [18]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS as english_stop_words

nlp_en = spacy.load("en_core_web_sm")


# Initialiser un set pour le vocabulaire anglais
vocab_en = set()

for title in data_en['title']:
    doc = nlp_en(title)
    for token in doc:
        lemma = token.lemma_.lower()
        # garder seulement les mots alphabétiques, minuscules, non stopwords
        if token.is_alpha and lemma not in english_stop_words:
            vocab_en.add(lemma)

print("Nombre de mots uniques en anglais :", len(vocab_en))


Nombre de mots uniques en anglais : 473


In [19]:
# Créons une colonne titre_clean en les mettant en miniscule sans stop word et en lemme 
def clean_title_fr(text):
    doc = nlp(text)
    return " ".join([token.lemma_.lower() 
                     for token in doc 
                     if token.is_alpha and token.lemma_.lower() not in french_stop_words])

# Appliquer sur la colonne 'title'
data_fr['title_clean'] = data_fr['title'].apply(clean_title_fr)

C:\Users\HP\AppData\Local\Temp\ipykernel_23704\4166352686.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fr['title_clean'] = data_fr['title'].apply(clean_title_fr)


In [43]:
# Construction de la matrice TF-IDF pour les articles en français en utilisant seulement les titres
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_fr = TfidfVectorizer(
    vocabulary=vocab_fr,  # on utilise le vocabulaire extrait précédemment
    max_df=0.8,                 # optionnel, mais sécuritaire
    min_df=2,                   # optionnel, mais sécuritaire
    ngram_range=(1, 2)          # unigrams et bigrams
)

# Construction de la matrice TF-IDF
tfidf_matrix_fr = vectorizer_fr.fit_transform(data_fr['title_clean'])
print("TF-IDF matrix shape (fr):", tfidf_matrix_fr.shape)


TF-IDF matrix shape (fr): (1133, 1822)


In [ ]:
"""
# Commençons avec les articles en français
# On utilise en premier le k-means 
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

inertia = []
K = range(2, 14)

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(tfidf_matrix_fr)
    inertia.append(kmeans.inertia_)

plt.plot(K, inertia, marker='o')
plt.xlabel("Nombre de clusters")
plt.ylabel("Inertie")
plt.title("Méthode du coude")
plt.show() """

In [44]:
# Application du K-means 
from sklearn.cluster import KMeans
k = 10
kmeans = KMeans(n_clusters=k, random_state=42)
data_fr["cluster"] = kmeans.fit_predict(tfidf_matrix_fr)

C:\Users\HP\AppData\Local\Temp\ipykernel_23704\2514362204.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fr["cluster"] = kmeans.fit_predict(tfidf_matrix_fr)


In [45]:
terms = vectorizer_fr.get_feature_names_out()

In [46]:
import numpy as np

def get_top_words_per_cluster(model, terms, n_words=10):
    for i, centroid in enumerate(model.cluster_centers_):
        top_indices = centroid.argsort()[-n_words:][::-1]
        top_terms = [terms[ind] for ind in top_indices]
        print(f"Cluster {i} : {', '.join(top_terms)}")
get_top_words_per_cluster(kmeans, terms)

Cluster 0 : ontologie, alignement, sémantique, construction, texte, base, annotation, raisonnement, guider, entrer
Cluster 1 : fréquent, incrémental, motif, itemset, gpu, apprentissage, graduels, temporalité, dépendance, fermés
Cluster 2 : aide, multidimensionnel, décision, donnée, fusion, navigation, xml, logiciel, acquisition, information
Cluster 3 : donnée, fouille, flux, base, classification, visualisation, entrepôt, relationnel, ensemble, textuel
Cluster 4 : réseau, social, dynamique, bayésiens, lien, information, communauté, sociaux, apprentissage, centralité
Cluster 5 : séquence, temporel, série, spatio, événement, motif, classification, dynamique, protéique, intervalle
Cluster 6 : règle, association, génération, classification, exception, comparaison, recherche, entrer, fouille, qualité
Cluster 7 : connaissance, sémantique, classification, document, apprentissage, graphe, automatique, image, recherche, texte
Cluster 8 : superviser, non, classification, apprentissage, sélection,

In [47]:
for i in range(k):
    print(f"\n### Cluster {i}")
    print(data_fr[data_fr["cluster"] == i]["title"].head(3).values)


### Cluster 0
["Contraintes prescriptives compatibles avec OWL2-ER pour évaluer la complétude d'ontologies"
 "L'ontologie OntoBiotope pour l'étude de la biodiversité microbienne"
 "Vers une instance française de NELL : chaîne TLN multilingue et modélisation d'ontologie"]

### Cluster 1
['Fouille de Motifs Graduels Fermés Fréquents Sous Contrainte de la Temporalité'
 "Une Approche d'Extraction de Motifs Graduels (Fermés) Fréquents Sous Contrainte de la Temporalité"
 'Défi EGC 2016 : Analyse par Motifs Fréquents et Topic Modeling']

### Cluster 2
["Une méthode pour l'estimation désagrégée de données de population à l'aide de données ouvertes"
 "Anonymiser des données multidimensionnelles à l'aide du coclustering"
 "Classification parcimonieuse pour l'aide à la reconnaissance de cibles radar"]

### Cluster 3
['Apport de la fouille de données pour la prévention du risque suicidaire'
 'Classification de Données Complexes par Globalisation de Mesures de Similarité via les Moyennes Quasi-Ari